In [3]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

from tempForecast.components.data_transformation import WeatherDatabase
from tempForecast.components.data_ingestion import weather_scraping

In [4]:
data = weather_scraping().load_data()
# on instancie la classe avec la base de donnée déjà chargé pour ne pas rescrapé tout
# si on met last_df en argument cela va scrapper seulement depuis la dernière date du df donc enlever pour éviter une interpolation
data_ingestion = weather_scraping(last_df=data)
# scraping 

data = data_ingestion.web_scraping()
# il faut ajouter cette base à la db
data_ingestion.load_data(df=data)
data = WeatherDatabase().load_data_and_transform()

2023-11-27 19:34:34.587566
https://www.infoclimat.fr/observations-meteo/archives/27/novembre/2023/paris-5eme-tour-zamansky-jussieu/000BV.html?dixminutes


c:\users\zbook create g7\desktop\temperature forecasting\src\tempForecast\components\data_ingestion.py:65: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  date_text = BeautifulSoup(


In [5]:
#data["unique_id"] = "temp"
data.rename(columns={"température":"y"}, inplace=True)
#data.rename(columns={"datetime":"ds"}, inplace=True)

data['y'].fillna(method='bfill', inplace=True)
data['vent'].fillna(method='bfill', inplace=True)
data['rafales'].fillna(0, inplace=True)
data['vent_direction'].fillna(0, inplace=True)
data['pt_de_rosee'].fillna(0, inplace=True)
data['humidite'].fillna(0, inplace=True)


C:\Users\Zbook Create G7\AppData\Local\Temp\ipykernel_50164\3164910419.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['y'].fillna(method='bfill', inplace=True)
C:\Users\Zbook Create G7\AppData\Local\Temp\ipykernel_50164\3164910419.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['vent'].fillna(method='bfill', inplace=True)


In [6]:
data.drop(columns=['bio_meteo','radiation_solaire'], inplace=True)
data.drop(columns='id', inplace=True)

In [7]:
data.rename(columns={"datetime":"ds"}, inplace=True)

In [12]:
data['unique_id'] = 'temp'

In [14]:

sf = StatsForecast(
    models = [AutoARIMA(season_length = 12)],
    freq = '10T'
)

sf.fit(data[['y','ds','unique_id']])
sf.predict(h=12, level=[95])


KeyboardInterrupt: 